<a href="https://colab.research.google.com/github/ladsong/if697-2020.2-data-science/blob/projeto2/Projeto_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Preparando o ambiente

In [30]:
import pandas as pd
import numpy as np
import re
import gc
import warnings
import sys

In [14]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [15]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle"
# /content/gdrive/My Drive/Kaggle é o caminho onde o arquivo kaggle.json está presente do Google Drive
#Mudar o diretorio
%cd /content/gdrive/My Drive/Kaggle

/content/gdrive/My Drive/Kaggle


##Carregando o dataset e processando os dados

In [16]:
df = pd.read_csv('project1_output.csv')

Vamos utilizar um subset do nosso dataset para diminuir o tempo de experimentação

In [17]:

df = df[:1000]
df = df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
df.dtypes

year                   int64
acousticness         float64
artists_data          object
danceability         float64
duration_ms            int64
energy               float64
explicit               int64
id                    object
instrumentalness     float64
loudness             float64
name                  object
popularity           float64
release_date          object
speechiness          float64
tempo                float64
first_artist          object
artists_by_artist     object
first_genre           object
dtype: object

Tirando colunas desnecessárias, pois só queremos trabalhar com valores numéricos

In [18]:
df = df.select_dtypes(exclude=['object'])

In [19]:
df.columns

Index(['year', 'acousticness', 'danceability', 'duration_ms', 'energy',
       'explicit', 'instrumentalness', 'loudness', 'popularity', 'speechiness',
       'tempo'],
      dtype='object')



####Escolher coluna para predição

In [20]:
target_col = df['popularity']

In [21]:
target_col

0      0.04
1      0.02
2      0.04
3      0.00
4      0.01
       ... 
995    0.00
996    0.00
997    0.00
998    0.00
999    0.00
Name: popularity, Length: 1000, dtype: float64

In [22]:
df = df.drop(columns=['popularity'])

##Separando os dados em teste e predição

Usando uma base 60/20/20

In [23]:
def get_x_data():
    # input 
    train, val, test = np.split(df.sample(frac=1), [int(.6*len(df)), int(.8*len(df))])
    
    return train, val, test

In [24]:
def get_y_data():
    # output
    train_labels, val_labels, test_labels = (
        np.split(
            target_col, 
            [int(.6*len(target_col)), int(.8*len(target_col))])
    )
    
    return train_labels, val_labels, test_labels

##Escolher os 4 algoritmos para predição

Vamos usar:



*   Regressão Linear
*   Multilayer perceptron
*   random forests
*   Gradient boost com lightgbm




In [2]:
!pip install mlflow --quiet

     |████████████████████████████████| 14.4 MB 58 kB/s 
     |████████████████████████████████| 1.1 MB 27.1 MB/s 
     |████████████████████████████████| 79 kB 7.2 MB/s 
     |████████████████████████████████| 170 kB 57.2 MB/s 
     |████████████████████████████████| 146 kB 39.1 MB/s 
     |████████████████████████████████| 56 kB 3.7 MB/s 
     |████████████████████████████████| 636 kB 37.7 MB/s 
     |████████████████████████████████| 75 kB 3.6 MB/s 
     |████████████████████████████████| 52 kB 1.1 MB/s 
     |████████████████████████████████| 63 kB 1.4 MB/s 


In [4]:
!pip install optuna

     |████████████████████████████████| 302 kB 25.3 MB/s 
     |████████████████████████████████| 80 kB 7.5 MB/s 
     |████████████████████████████████| 111 kB 37.5 MB/s 
     |████████████████████████████████| 141 kB 56.5 MB/s 
     |████████████████████████████████| 49 kB 4.7 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=ce5583930f9674fdda11cc41c652383fe2a3172f2f2bdc274bc583f8c3f3bea5
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [25]:
import mlflow
import mlflow.sklearn

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import mixed_precision
from tensorflow.keras.layers.experimental import preprocessing

import optuna

from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    r2_score,
    confusion_matrix,
    classification_report,
    accuracy_score
)
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

import lightgbm
from lightgbm import LGBMRegressor

Função de avaliação das metricas

In [6]:
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

In [7]:
mlflow.sklearn.autolog()
mlflow.tensorflow.autolog()
mlflow.lightgbm.autolog()

2021/08/16 22:49:00 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of lightgbm. If you encounter errors during autologging, try upgrading / downgrading lightgbm to a supported version, or try upgrading MLflow.


###Regressão Linear

In [33]:
def linear_regression(trial):
    train, test, val = get_x_data()
    train_labels, val_labels, test_labels = get_y_data()
    
    warnings.filterwarnings("ignore")
    np.random.seed(40)

    # Start an MLflow run; the "with" keyword ensures we'll close the run even if this cell crashes
    with mlflow.start_run(run_name="Linear Regression"):
        reg = LinearRegression()
        reg.fit(train, train_labels)

        predictions = reg.predict(val)

        (rmse, mae, r2) = eval_metrics(val_labels, predictions)

        # Print out model metrics
        print("Modelo de regressão linear")
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        # Log mlflow attributes for mlflow UI
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)
        
        
        gc.collect()
        
        return rmse

In [34]:
study = optuna.create_study()
study.optimize(linear_regression, n_trials=1)

[I 2021-08-16 22:53:35,524] A new study created in memory with name: no-name-7934c31b-d81f-4a4c-9736-23f6547d04d0


Modelo de regressão linear
  RMSE: 0.019455898528420473
  MAE: 0.017646177370942265
  R2: -57.801085444379645


[I 2021-08-16 22:53:36,137] Trial 0 finished with value: 0.019455898528420473 and parameters: {}. Best is trial 0 with value: 0.019455898528420473.


Multilayer Perceptron

In [35]:
def mlp(trial):
    train, test, val = get_x_data()
    train_labels, val_labels, test_labels = get_y_data()
    
    # hyper-parameters to test
    params = {
        "hidden_units": trial.suggest_int("hidden_units", 3, 15),
        "lr": trial.suggest_float("lr", 1e-5, 1e-3, log=True),
        "epochs": trial.suggest_int("epochs", 10, 50)
    }
    
    warnings.filterwarnings("ignore")
    np.random.seed(40)
    
    # Start an MLflow run
    with mlflow.start_run(run_name="MLP"):
        normalizer = preprocessing.Normalization(axis=-1)
        normalizer.adapt(np.array(train))
        
        mlp_model = tf.keras.Sequential([
            normalizer,
            layers.Dense(units=params["hidden_units"]),
            layers.Dense(units=params["hidden_units"]),
            layers.Dense(units=params["hidden_units"]),
            layers.Dense(units=1),
        ])

        mlp_model.summary()
        
        mlp_model.compile(
            optimizer=tf.optimizers.Adam(learning_rate=params["lr"]),
            loss='mean_squared_error'
        )

        history = mlp_model.fit(
            train, train_labels,
            validation_data=(test, test_labels),
            epochs=params["epochs"]
        )
        
        predictions = mlp_model.predict(val)

        (rmse, mae, r2) = eval_metrics(val_labels, predictions)

        # Print out model metrics
        print("MLP model")
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        # Log mlflow attributes for mlflow UI
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)
        mlflow.log_params(trial.params)
        mlflow.set_tags(
            {
                "estimator_name":"MultiLayerPerceptron",
                "estimator_class":"Keras"
            }
        )
        #mlflow.tensorflow.log_model(mlp_model, "model")
        #modelpath = "./mlflow/freight_value/model-mlp"
        #mlflow.tensorflow.save_model(mlp_model, modelpath)
        tf.keras.backend.clear_session()

        gc.collect()
        
        return rmse

In [36]:

study = optuna.create_study()
study.optimize(mlp, n_trials=10)

[I 2021-08-16 22:55:09,963] A new study created in memory with name: no-name-d38c4de0-67eb-4b48-96a1-d921d304a6c5


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
normalization (Normalization (None, 10)                21        
_________________________________________________________________
dense (Dense)                (None, 7)                 77        
_________________________________________________________________
dense_1 (Dense)              (None, 7)                 56        
_________________________________________________________________
dense_2 (Dense)              (None, 7)                 56        
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 8         
Total params: 218
Trainable params: 197
Non-trainable params: 21
_________________________________________________________________
Epoch 1/20
19/19 [==============================] - 1s 18ms/step - loss: 2.7717 - val_loss: 1.8166
Epoch 2/20
19/19 [======

[I 2021-08-16 22:55:17,696] Trial 0 finished with value: 0.6354928533006559 and parameters: {'hidden_units': 7, 'lr': 0.00011080699654335682, 'epochs': 20}. Best is trial 0 with value: 0.6354928533006559.


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
normalization (Normalization (None, 10)                21        
_________________________________________________________________
dense (Dense)                (None, 12)                132       
_________________________________________________________________
dense_1 (Dense)              (None, 12)                156       
_________________________________________________________________
dense_2 (Dense)              (None, 12)                156       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 13        
Total params: 478
Trainable params: 457
Non-trainable params: 21
_________________________________________________________________
Epoch 1/15
19/19 [==============================] - 1s 16ms/step - loss: 0.8488 - val_loss: 0.5878
Epoch 2/15
19/19 [======

[I 2021-08-16 22:55:22,657] Trial 1 finished with value: 0.6770997652117401 and parameters: {'hidden_units': 12, 'lr': 1.7116700104715165e-05, 'epochs': 15}. Best is trial 0 with value: 0.6354928533006559.


...

[I 2021-08-16 22:56:07,624] Trial 7 finished with value: 0.019510929578675073 and parameters: {'hidden_units': 12, 'lr': 0.00019891917216085112, 'epochs': 49}. Best is trial 6 with value: 0.01909312589403764.


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
normalization (Normalization (None, 10)                21        
_________________________________________________________________
dense (Dense)                (None, 11)                121       
_________________________________________________________________
dense_1 (Dense)              (None, 11)                132       
_________________________________________________________________
dense_2 (Dense)              (None, 11)                132       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 12        
Total params: 418
Trainable params: 397
Non-trainable params: 21
_________________________________________________________________
Epoch 1/24
19/19 [==============================] - 1s 17ms/step - loss: 0.7211 - val_loss: 0.4677
Epoch 2/24
19/19 [======

[I 2021-08-16 22:56:13,409] Trial 8 finished with value: 0.06935200655981462 and parameters: {'hidden_units': 11, 'lr': 0.0004198674582428264, 'epochs': 24}. Best is trial 6 with value: 0.01909312589403764.


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
normalization (Normalization (None, 10)                21        
_________________________________________________________________
dense (Dense)                (None, 9)                 99        
_________________________________________________________________
dense_1 (Dense)              (None, 9)                 90        
_________________________________________________________________
dense_2 (Dense)              (None, 9)                 90        
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 10        
Total params: 310
Trainable params: 289
Non-trainable params: 21
_________________________________________________________________
Epoch 1/17
19/19 [==============================] - 1s 18ms/step - loss: 2.1980 - val_loss: 1.3545
Epoch 2/17
19/19 [======

[I 2021-08-16 22:56:18,664] Trial 9 finished with value: 0.34464425675847316 and parameters: {'hidden_units': 9, 'lr': 0.00032911589336186647, 'epochs': 17}. Best is trial 6 with value: 0.01909312589403764.


###Random Forest

In [39]:
def random_forest(trial):
    train, test, val = get_x_data()
    train_labels, val_labels, test_labels = get_y_data()
    
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 150),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 5),
    }
    
    warnings.filterwarnings("ignore")
    np.random.seed(40)
    
    with mlflow.start_run(run_name="Random Forest"):
        rf = RandomForestRegressor(
            max_depth=params["max_depth"],
            n_estimators=params["n_estimators"],
            min_samples_split=params["min_samples_split"],
            random_state=0
        )
        rf.fit(train, train_labels)
        
        predictions = rf.predict(val)
        
        (rmse, mae, r2) = eval_metrics(val_labels, predictions)
        
        print("Random Forest model")
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)
        
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)
        mlflow.log_params(trial.params)
        
        gc.collect()
        
        return rmse

In [40]:
study = optuna.create_study()
study.optimize(random_forest, n_trials=10)

[I 2021-08-16 22:57:52,265] A new study created in memory with name: no-name-7e88c718-f0c5-4b95-aaa6-a2cb437be7f2


Random Forest model
  RMSE: 0.03076379686953024
  MAE: 0.019045805482472918
  R2: -146.01533170170387


[I 2021-08-16 22:57:53,410] Trial 0 finished with value: 0.03076379686953024 and parameters: {'n_estimators': 59, 'max_depth': 6, 'min_samples_split': 4}. Best is trial 0 with value: 0.03076379686953024.


Random Forest model
  RMSE: 0.031478965602240096
  MAE: 0.02007207003657248
  R2: -152.9301398659441


[I 2021-08-16 22:57:54,740] Trial 1 finished with value: 0.031478965602240096 and parameters: {'n_estimators': 99, 'max_depth': 8, 'min_samples_split': 2}. Best is trial 0 with value: 0.03076379686953024.


...

[I 2021-08-16 22:58:02,087] Trial 7 finished with value: 0.03232207083849229 and parameters: {'n_estimators': 127, 'max_depth': 9, 'min_samples_split': 5}. Best is trial 5 with value: 0.02855276636257497.


Random Forest model
  RMSE: 0.032979007299821846
  MAE: 0.02042099620748074
  R2: -167.9498908709441


[I 2021-08-16 22:58:03,404] Trial 8 finished with value: 0.032979007299821846 and parameters: {'n_estimators': 92, 'max_depth': 10, 'min_samples_split': 5}. Best is trial 5 with value: 0.02855276636257497.


Random Forest model
  RMSE: 0.03235252028314271
  MAE: 0.020362194924787847
  R2: -161.59193299746187


[I 2021-08-16 22:58:04,921] Trial 9 finished with value: 0.03235252028314271 and parameters: {'n_estimators': 139, 'max_depth': 10, 'min_samples_split': 2}. Best is trial 5 with value: 0.02855276636257497.


###Gradient boost com lightgbm

In [41]:
def gradient_boosting(trial):
    train, test, val = get_x_data()
    train_labels, val_labels, test_labels = get_y_data()
    
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 150),
        "num_leaves": trial.suggest_int("num_leaves", 25, 35),
        "max_depth": trial.suggest_int("max_depth", 3, 10)
    }
    
    warnings.filterwarnings("ignore")
    np.random.seed(40)
    
    with mlflow.start_run(run_name="Gradient Boosting"):
#         model = LGBMRegressor(
#             max_depth=params["max_depth"],
#             n_estimators=params["n_estimators"],
#             num_leaves=params["num_leaves"],
#         )
        model = XGBRegressor(
            max_depth=params["max_depth"],
            n_estimators=params["n_estimators"],
        )
        model.fit(train, train_labels)
        
        predictions = model.predict(test)
        print('Prediction: %.3f' % predictions[0])
        
        (rmse, mae, r2) = eval_metrics(val_labels, predictions)

        print("LGBM model")
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        # Log mlflow attributes for mlflow UI
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)
        mlflow.log_params(trial.params)
        mlflow.set_tags(
            {
                "estimator_class":"LightGBM",
                "estimator_name":"Gradient Boosting"
            }
        )
        mlflow.sklearn.log_model(model, "model")
        
        gc.collect()
        
        return rmse

In [42]:
study = optuna.create_study()
study.optimize(gradient_boosting, n_trials=10)

[I 2021-08-16 23:00:25,969] A new study created in memory with name: no-name-987ccfdd-2347-4d05-9a79-b48875b7dfaf


[23:00:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Prediction: 0.004
LGBM model
  RMSE: 0.04059447506877251
  MAE: 0.023190308749675755
  R2: -254.98623784220482


[I 2021-08-16 23:00:26,744] Trial 0 finished with value: 0.04059447506877251 and parameters: {'n_estimators': 143, 'num_leaves': 26, 'max_depth': 5}. Best is trial 0 with value: 0.04059447506877251.


[23:00:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Prediction: 0.006
LGBM model
  RMSE: 0.039719294511375125
  MAE: 0.022940721303224568
  R2: -244.06755052137538


[I 2021-08-16 23:00:27,586] Trial 1 finished with value: 0.039719294511375125 and parameters: {'n_estimators': 124, 'num_leaves': 26, 'max_depth': 5}. Best is trial 1 with value: 0.039719294511375125.


...

[I 2021-08-16 23:00:32,360] Trial 7 finished with value: 0.03580880262949954 and parameters: {'n_estimators': 102, 'num_leaves': 35, 'max_depth': 9}. Best is trial 5 with value: 0.03402010992262072.


[23:00:32] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Prediction: 0.006
LGBM model
  RMSE: 0.03995935384494424
  MAE: 0.023054923653602604
  R2: -247.0388286921104


[I 2021-08-16 23:00:33,089] Trial 8 finished with value: 0.03995935384494424 and parameters: {'n_estimators': 131, 'num_leaves': 31, 'max_depth': 5}. Best is trial 5 with value: 0.03402010992262072.


[23:00:33] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Prediction: 0.003
LGBM model
  RMSE: 0.040808810350956896
  MAE: 0.023329358160495754
  R2: -257.69654404044536


[I 2021-08-16 23:00:33,960] Trial 9 finished with value: 0.040808810350956896 and parameters: {'n_estimators': 143, 'num_leaves': 31, 'max_depth': 4}. Best is trial 5 with value: 0.03402010992262072.
